# Loading Data

In [100]:
import pandas as pd
import numpy as np
import streamlit as st
from matplotlib import pyplot as plt
import seaborn as sns
import plotly.express as px

In [101]:
data = pd.read_csv('kc_house_data.csv')

## Limpeza dos dados

In [102]:
## Alterando o formato de data

data['date'] = pd.to_datetime(data['date'])
data['year'] = data['date'].dt.year
data['month'] = data['date'].dt.month

## Removendo dados duplicados 

data = data.drop_duplicates(subset=['id'], keep='last')

## Criando uma nova condição para os imóveis
- Se o valor do imóvel for maior que 540000 ele é high_level
- Se o valor do imóvel for menor que 540000 ele é low_level

In [103]:
data['level'] = 'standard'
data.loc[data['price'] > 540000, 'level'] = 'high_level'
data.loc[data['price'] < 540000, 'level'] = 'low_level'

## Criando uma nova coluna pela idade do imóvel 
- Se o valor da coluna “date” for maior que 2014-01-01 => ‘new_house’
- Se o valor da coluna “date” for menor que 2014-01-01 => ‘old_house’

## Criando uma nova coluna para "Estação"

In [104]:
 data['season'] = data['month'].apply(lambda x: 'summer' if (x > 5) & (x < 8) else
    'spring' if (x > 2) & (x < 5) else
    'fall' if (x > 8) & (x < 12) else
    'winter')

In [105]:
data['house_age'] = data['date'].apply ( lambda x: 'new_house' 
                                        if x > pd.to_datetime('2014', format='%Y') 
                                        else 'old_house')

## Criando uma nova coluna de porão

In [106]:
data['basement'] = data['sqft_basement'].apply (lambda x: 'nao' if x == 0 else
                                             'sim')

## Delete as colunas: “sqft_living15”, “sqft_lot15” e "house_age"

In [107]:
data = data.drop(['sqft_living15', 'sqft_lot15','house_age'], axis=1)

## Modificando o TIPO a Coluna “yr_built” para DATE

In [108]:
data['yr_built'] = pd.to_datetime(data['yr_built'], format='%Y')

## Modificando o TIPO a Coluna “yr_renovated” para DATE

In [109]:
data['yr_renovated'] = data['yr_renovated'].apply(
    lambda x: pd.to_datetime('1970', format='%Y')
    if x == 0 else pd.to_datetime(x, format='%Y'))

## Modificando um outlier de 33 quartos assumindo ser um erro de digitação para 3 quartos

In [110]:
data.loc[data['bedrooms'] == 33, 'bedrooms'] = 3

# EDA

### H1 - Casas com vista pra água são 30% mais caras, na média

In [111]:
# Descobrir a média de preço das casas com vista pro mar

vista_pra_água = np.round(data.loc[data['waterfront'] ==1, 'price'].mean(),2)
print ('O preço médio de imóveis com vista pra água é %.2f' %vista_pra_água)

O preço médio de imóveis com vista pra água é 1661876.02


In [112]:
imóveis_sem_vista_pra_água = np.round(data.loc[data['waterfront'] ==0, 'price'].mean(),2)
print ('O preço médio de imóveis sem vista pra água é %.2f' %imóveis_sem_vista_pra_água)

O preço médio de imóveis sem vista pra água é 533066.46


- A H1 é verdadeira

### H2 - Imóveis com data de construção menor que 1955, são 50% mais baratos, na média.

In [113]:
casas_de_1995 = data.loc[data['yr_built'] < '1955-01-01' , 'price'].mean()
print ('A media dos imóveis é de %.2f'%casas_de_1995)

A media dos imóveis é de 540100.38


In [114]:
casas_depois_1995 = data.loc[data['yr_built'] > '1955-01-01' , 'price'].mean()
print ('A media dos imóveis é de %.2f'%casas_depois_1995)

A media dos imóveis é de 543873.83


- A H2 é falsa

### H3 - Imóveis sem porão que possuem sqrt_lot, são 50% maiores do que com porão.

In [115]:
h3 = data[['sqft_lot','basement']].groupby('basement').mean().reset_index()
print (h3)

  basement      sqft_lot
0      nao  16325.928774
1      sim  13295.994537


- A H3 é falsa pois os imóveis sem porão não são 50% maiores que os com porão

### H4 - O crescimento do preço dos imóveis YoY ( Year over Year ) é de 10%

In [116]:
h4 = data[['year','price']].groupby('year').mean().reset_index()

In [117]:
print(h4)

   year          price
0  2014  541334.572732
1  2015  542304.265777


- H4 é falsa pois o YoY é de menos de 1% em média

### H5 - Imóveis com 3 banheiros tem um crescimento MoM ( Month over Month ) de 15%

In [118]:
h5 = data[['month','price']].loc[data['bathrooms'] == 3].groupby('month').mean().reset_index()
print(h5)

    month          price
0       1  672923.170732
1       2  696714.422222
2       3  768495.096154
3       4  709751.988235
4       5  691097.629213
5       6  764084.285714
6       7  741405.042857
7       8  743506.451613
8       9  650464.700000
9      10  646302.353846
10     11  679910.465116
11     12  728341.187500


- H5 é falsa pois não há um aumento de 15% MoM nos imóveis com 3 banheiros

# Questões do CEO
- Quais imóveis devo comprar e qual valor devo pagar por eles?
- Quando devo vende-lo e por quanto devo vender?


## Questão 1 

In [119]:
data['sqrt_price'] = data['price']/data['sqft_lot']

In [121]:
df = data[['zipcode','sqrt_price']].groupby('zipcode').median().reset_index()
df.columns=['zipcode','price_zipcode']
df

,zipcode,price_zipcode
0,98001,27.248677
1,98002,30.223655
2,98003,30.123342
3,98004,94.998653
4,98005,55.622924
...,...,...
65,98177,60.063652
66,98178,38.844792
67,98188,30.177515
68,98198,31.250000


In [122]:
df1 = pd.merge(data, df, on='zipcode', how='inner')
df1

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,zipcode,lat,long,year,month,level,season,basement,sqrt_price,price_zipcode
0,7129300520,2014-10-13,221900.0,3,1.00,1180,5650,1.0,0,0,...,98178,47.5112,-122.257,2014,10,low_level,fall,nao,39.274336,38.844792
1,4060000240,2014-06-23,205425.0,2,1.00,880,6780,1.0,0,0,...,98178,47.5009,-122.248,2014,6,low_level,summer,nao,30.298673,38.844792
2,4058801670,2014-07-17,445000.0,3,2.25,2100,8201,1.0,0,2,...,98178,47.5091,-122.244,2014,7,low_level,summer,sim,54.261675,38.844792
3,2976800796,2014-09-25,236000.0,3,1.00,1300,5898,1.0,0,0,...,98178,47.5053,-122.255,2014,9,low_level,fall,nao,40.013564,38.844792
4,6874200960,2015-02-27,170000.0,2,1.00,860,5265,1.0,0,0,...,98178,47.5048,-122.272,2015,2,low_level,winter,nao,32.288699,38.844792
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21431,2525049086,2014-10-03,2720000.0,4,3.25,3990,18115,2.0,0,0,...,98039,47.6177,-122.229,2014,10,high_level,fall,nao,150.151808,115.777805
21432,2525049113,2014-07-25,1950000.0,4,3.50,4065,18713,2.0,0,0,...,98039,47.6209,-122.237,2014,7,high_level,summer,nao,104.205632,115.777805
21433,3262300485,2015-04-21,2250000.0,5,5.25,3410,8118,2.0,0,0,...,98039,47.6295,-122.236,2015,4,high_level,spring,nao,277.161863,115.777805
21434,6447300365,2014-11-13,2900000.0,5,4.00,5190,14600,2.0,0,1,...,98039,47.6102,-122.225,2014,11,high_level,fall,nao,198.630137,115.777805


In [137]:
# Criando a coluna "to_buy"
df1['to_buy'] = df1[['sqrt_price','price_zipcode','condition']].apply(lambda x: 'buy' if (x['sqrt_price'] < x['price_zipcode']) & (x['condition']>=3)
                                                          else 'not_buy', axis=1)

In [124]:
df1[['condition','to_buy']]

,condition,to_buy
0,3,not_buy
1,4,buy
2,3,not_buy
3,3,not_buy
4,3,buy
...,...,...
21431,4,not_buy
21432,4,buy
21433,3,not_buy
21434,3,not_buy


In [125]:
df1[df1['to_buy']=='buy'].shape[0]

10538

- 1. O CEO pode comprar 10538 imóveis.

# Questão 2

In [126]:
resell_season = df1[['sqrt_price','zipcode','season']].groupby(['zipcode','season']).median().reset_index()
resell_season.columns = ['zipcode','season','price_to_sell']

In [127]:
df2 = pd.merge(df1, resell_season, on=['zipcode','season'])
df2

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,long,year,month,level,season,basement,sqrt_price,price_zipcode,to_buy,price_to_sell
0,7129300520,2014-10-13,221900.0,3,1.00,1180,5650,1.0,0,0,...,-122.257,2014,10,low_level,fall,nao,39.274336,38.844792,not_buy,39.427507
1,2976800796,2014-09-25,236000.0,3,1.00,1300,5898,1.0,0,0,...,-122.255,2014,9,low_level,fall,nao,40.013564,38.844792,not_buy,39.427507
2,1180003090,2014-09-06,190000.0,2,1.00,630,6000,1.0,0,0,...,-122.221,2014,9,low_level,fall,nao,31.666667,38.844792,buy,39.427507
3,2171400197,2014-09-18,350000.0,5,3.00,2520,5500,1.0,0,0,...,-122.255,2014,9,low_level,fall,sim,63.636364,38.844792,not_buy,39.427507
4,1180002378,2014-09-26,299000.0,4,2.50,1950,3000,2.0,0,0,...,-122.226,2014,9,low_level,fall,nao,99.666667,38.844792,not_buy,39.427507
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21431,3262300555,2014-07-08,2458000.0,4,5.25,6500,14986,2.0,0,0,...,-122.236,2014,7,high_level,summer,sim,164.019752,115.777805,not_buy,94.358870
21432,2525049263,2014-07-09,2680000.0,5,3.00,4290,20445,2.0,0,0,...,-122.239,2014,7,high_level,summer,nao,131.083394,115.777805,not_buy,94.358870
21433,5427100150,2014-06-26,1410000.0,4,2.25,3250,16684,2.0,0,0,...,-122.229,2014,6,high_level,summer,nao,84.512107,115.777805,buy,94.358870
21434,2425049066,2014-06-16,1920000.0,4,2.50,3070,34412,1.0,0,3,...,-122.240,2014,6,high_level,summer,sim,55.794490,115.777805,buy,94.358870


In [128]:
df2['season_to_sell'] = np.round(
    df2[['sqrt_price', 'price_to_sell',
         'sqft_lot']].apply(lambda x:
                            (x['price_to_sell'] * x['sqft_lot'] * 1.3)
                            if x['price_to_sell'] > x['sqrt_price'] else
                            (x['price_to_sell'] * x['sqft_lot'] * 1.1),
                            axis=1), 2)

In [129]:
df2[df2['to_buy']=='buy']

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,year,month,level,season,basement,sqrt_price,price_zipcode,to_buy,price_to_sell,season_to_sell
2,1180003090,2014-09-06,190000.0,2,1.00,630,6000,1.0,0,0,...,2014,9,low_level,fall,nao,31.666667,38.844792,buy,39.427507,307534.55
6,179000350,2014-11-05,194000.0,3,1.50,1010,5000,1.0,0,0,...,2014,11,low_level,fall,nao,38.800000,38.844792,buy,39.427507,256278.79
7,3810000202,2014-09-05,251700.0,3,2.25,1810,11800,1.0,0,0,...,2014,9,low_level,fall,sim,21.330508,38.844792,buy,39.427507,604817.95
8,4058802105,2014-09-04,150000.0,3,1.00,1450,6776,1.0,0,0,...,2014,9,low_level,fall,nao,22.136954,38.844792,buy,39.427507,347309.02
9,7878400135,2014-11-20,355000.0,3,2.25,2550,9674,1.0,0,0,...,2014,11,low_level,fall,sim,36.696299,38.844792,buy,39.427507,495848.21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21428,3262301355,2014-07-25,1320000.0,3,2.75,2680,20104,1.0,0,0,...,2014,7,high_level,summer,sim,65.658575,115.777805,buy,94.358870,2466087.94
21429,5427110040,2014-06-09,1225000.0,4,2.50,2740,16007,2.0,0,0,...,2014,6,high_level,summer,nao,76.529019,115.777805,buy,94.358870,1963523.16
21433,5427100150,2014-06-26,1410000.0,4,2.25,3250,16684,2.0,0,0,...,2014,6,high_level,summer,nao,84.512107,115.777805,buy,94.358870,2046568.40
21434,2425049066,2014-06-16,1920000.0,4,2.50,3070,34412,1.0,0,3,...,2014,6,high_level,summer,sim,55.794490,115.777805,buy,94.358870,4221200.66


In [130]:
df2['new_price'] = np.round(df2['price_to_sell']* df2['sqft_lot'],2)

In [131]:
df2['profit']=df2['season_to_sell'] - df2['new_price']

In [132]:
df2['year']

0        2014
1        2014
2        2014
3        2014
4        2014
         ... 
21431    2014
21432    2014
21433    2014
21434    2014
21435    2014
Name: year, Length: 21436, dtype: int64

In [133]:
df2[df2['to_buy']=='buy']['profit'].sum()

3492823431.53

- Se o CEO comprar todos os imóveis nas condições de "buy" ele terá um lucro de 3492823431.53.

In [134]:
df2[['id','profit']].sort_values('profit', ascending=False)

,id,profit
17404,2624089007,25845477.84
8467,2623069031,16459860.70
18807,1020069017,11917307.45
1985,125069038,11025789.47
17654,3624079067,10998775.85
...,...,...
20293,1773101159,3716.87
8577,8562780280,3677.43
8568,8562780540,3631.46
8573,8562780430,3565.07


In [139]:
# Criando a coluna "percentage_profit"
df2['percentage_profit']=np.round(df2['profit']/df2['price'],2)